# Consideraciones Importantes

Hasta ahora se han creado dos contenedores, uno para la BD con MONGO, y otro para la API con FASTAPI. Sin embargo como se ha trabajado desde varios computadores, es importante tener en cuenta que la BD esta alojada en un Volumen de Docker, que se llama `dbAppTienda`. Si por alguna razon no se ha creado en un computador o en otro, crearla con datos de prueba. Y la manera correcta de levantar el servicio es:

```sh
docker run --detach --name tuTiendaDB --mount src=dbAppTienda,dst=/data/db --publish 27017:27017 --network platzinet mongo:5.0
```
Para evitar incompatibilidades, se usara de aqui en adelante *mongo 5.0*.

- `docker volume create dbAppTienda` si no se ha creado el volumen. 

Para la API, no se usara por ahora contenedor. Se iniciara directamente desde la consola `uvicorn`.

`uvicorn main:app --reload`

Se creara una nueva coleccion en la BD, llamada `db-tienda`, con una unica coleccion, llamada `clients`, ver `insert_clients.js` donde esta el script.

## Seguir avanzando

Para seguir avanzando en un codigo eficiente, introduciremos dos conceptos fundamentales, el primero es Eventos de Ciclo de Vida, y el segundo Declaracion Tipo Opcional

## Eventos de Ciclo de Vida

FastAPI, al igual que otros frameworks web basados en ASGI (como Starlette, en el que se basa FastAPI), proporciona "eventos de ciclo de vida" que te permiten ejecutar código en momentos específicos de la vida de tu aplicación.

Los eventos de ciclo de vida son un concepto fundamental en el desarrollo de software, especialmente en aplicaciones que manejan recursos externos o que necesitan realizar acciones en momentos específicos de su ejecución. No son exclusivos de FastAPI o Python; son una idea universal que encontrarás en la mayoría de los frameworks y lenguajes modernos.

### ¿Qué son los Eventos de Ciclo de Vida?

En términos generales, los eventos de ciclo de vida son ganchos o puntos de extensión predefinidos en la línea de tiempo de una aplicación, un componente, un servicio o incluso un objeto. Estos ganchos te permiten ejecutar código personalizado en fases clave, como:

- Inicialización: Cuando algo se crea o se pone en marcha.

- Inicio: Cuando un servicio o aplicación comienza a estar disponible para operar.

- Actualización/Cambio: Cuando el estado de algo cambia.

- Pausa/Suspensión: Cuando una operación se detiene temporalmente.

- Reanudación: Cuando una operación se reinicia después de una pausa.

- Destrucción/Apagado: Cuando algo finaliza su ejecución y libera recursos.

### Eventos Startup y Shutdown

#### Evento startup (Inicialización)

El evento `startup` en FastAPI se refiere a las acciones que tu aplicación debe realizar una única vez cuando se está iniciando, antes de que esté lista para procesar cualquier solicitud HTTP entrante.

##### ¿Cuándo se ejecuta?
- Cuando ejecutas tu aplicación FastAPI (por ejemplo, con ``uvicorn main:app --reload``).

- FastAPI detecta que el servidor se está levantando.

- Antes de que el servidor abra sus puertos y empiece a escuchar peticiones, ejecuta todas las funciones decoradas con ``@app.on_event("startup")``

Se usa comunmente para: extablecer conexiones a bases de datos, cargar configuraciones globales como variables de entorno, etc

#### Evento shutdown (Apagado)

El evento ``shutdown`` en FastAPI se refiere a las acciones que tu aplicación debe realizar una única vez cuando se está cerrando, antes de que el proceso termine por completo.

- Cuando el servidor FastAPI recibe una señal para apagarse (por ejemplo, presionas ``Ctrl+C`` en la terminal donde corre ``uvicorn``, o un orquestador como Docker/Kubernetes envía una señal de terminación).

- FastAPI detiene la aceptación de nuevas solicitudes.

- Antes de que el proceso de la aplicación finalice, ejecuta todas las funciones decoradas con ``@app.on_event("shutdown")``.

Se usan comunmente para cerrar conexiones a bases de datos, vaciar logs, liberarar recursos del sistema entre otros. 

En resumen, los eventos ``startup`` y ``shutdown`` son tus mejores amigos para manejar el ciclo de vida de los recursos de tu aplicación, garantizando que todo se inicialice correctamente y se limpie de forma segura.

## Declaración de Tipo Opcional

En programación, una Declaración de Tipo Opcional significa que una variable, un parámetro de función o un valor de retorno *puede ser de un tipo específico o puede ser "nada"* (generalmente representado por ``None`` en Python, ``null`` en JavaScript/Java/C#).

En Python, esto se expresa usando ``Optional[Tipo]`` (por ejemplo, ``Optional[int]``).

### ¿Por qué es Necesario en Cierto Tipo de Situaciones?

La Declaración de Tipo Opcional es esencial en situaciones donde la ausencia de un valor es una posibilidad legítima y debe ser manejada de forma explícita, no como un error inesperado. 

1. Recursos Externos que Podrían Fallar o No Estar Presentes: *Conexiones a Bases de Datos o APIs Externas:* Como tu ejemplo de ``db_client``. Una aplicación no puede asumir que siempre tendrá una conexión activa a la DB. La conexión puede no establecerse al inicio, o puede perderse durante la ejecución. El ``Optional`` te fuerza a verificar si la conexión existe antes de usarla.

2. Búsquedas de Datos (Cuando el Elemento Podría No Existir):Ejemplo: Buscar un usuario por ID. Si el ID no existe, la función debe retornar "nada" (e.g., ``None``), no un usuario incompleto o un error que cause un crasheo. El tipo de retorno de la función sería ``Optional[UserObject]``.
